<a href="https://colab.research.google.com/github/nw93929/DS2002FinalProject/blob/main/SCRIPTS/etl_pipeline_setup_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf DS2002FinalProject
!git clone https://github.com/nw93929/DS2002FinalProject.git
%cd DS2002FinalProject

Cloning into 'DS2002FinalProject'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 155 (delta 76), reused 116 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (155/155), 16.64 MiB | 30.33 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/DS2002FinalProject


In [ ]:
# Need to install necessary libraries
!pip install pandas openpyxl sqlalchemy pymysql google-cloud-storage

In [151]:
import pandas as pd

# Load necessary data
outcomes = pd.read_excel('Data/WinningPartyByCounties.xlsx')
demographics2012 = pd.read_excel('Data/demographicsswingstates2012.xlsx')
demographics2016 = pd.read_excel('Data/demographicsswingstates2016.xlsx')
demographics2020 = pd.read_excel('Data/demographicswingstates2020.xlsx')

# Clean outcomes data
outcomes['state'] = outcomes['state'].str.title()  # Standardize state names
outcomes['county_name'] = outcomes['county_name'].str.title()  # Standardize county names
columns_to_drop = ['office', 'version', 'county_fips', 'mode']
outcomes_cleaned = outcomes.drop(columns=columns_to_drop)

# Columns to keep for demographics, different for 2012/2020 and 2016
columns_to_keep = [
    'County Name', 'State Name', 'Females 18-24 %', 'Males 65+ %',
    'White alone %', 'Black or African American alone %',
    'less than $39,999 %', 'more than $200,000 %',
    'Regular high school diploma %', "Master's degree %",
    'In labor force %', 'Not in labor force %'
]
columns_to_keep_2016 = [
    'County Name', 'State Name', 'Females 18-24 %', 'Males 65+ %',
    'White alone %', 'Black or African American alone %',
    'less than $39,999 %', 'more than $200,000 %',
    'Regular high school diploma %', "Master's degree %",
    'In labor force Percentage %', 'Not in labor force Percentage %'
]

# Clean demographics data
demographics2012_cleaned = demographics2012[columns_to_keep].copy()
demographics2016_cleaned = demographics2016[columns_to_keep_2016].rename(
    columns={
        "In labor force Percentage %": "In labor force %",
        "Not in labor force Percentage %": "Not in labor force %"
    }
).copy()
demographics2020_cleaned = demographics2020[columns_to_keep].copy()

# Add 'year' column
demographics2012_cleaned['year'] = 2012
demographics2016_cleaned['year'] = 2016
demographics2020_cleaned['year'] = 2020

# Standardize column names for merging
for df in [demographics2012_cleaned, demographics2016_cleaned, demographics2020_cleaned]:
    df['county_name'] = (
        df['County Name']
        .str.replace(r'\s*County\s*$', '', regex=True)
        .str.strip()
        .str.title()
    )
    df['state'] = df['State Name'].str.strip().str.title()

# Merge datasets for each year
merged_2012 = pd.merge(
    outcomes_cleaned[outcomes_cleaned['year'] == 2012],
    demographics2012_cleaned,
    on=['county_name', 'state'],
    how='inner'
)
merged_2016 = pd.merge(
    outcomes_cleaned[outcomes_cleaned['year'] == 2016],
    demographics2016_cleaned,
    on=['county_name', 'state'],
    how='inner'
)
merged_2020 = pd.merge(
    outcomes_cleaned[outcomes_cleaned['year'] == 2020],
    demographics2020_cleaned,
    on=['county_name', 'state'],
    how='inner'
)

# Combine all years merged data
merged_data = pd.concat([merged_2012, merged_2016, merged_2020], axis=0)

# Clean up column names
merged_data = merged_data.rename(columns=lambda x: x.strip())
if 'year_y' in merged_data.columns:
    merged_data = merged_data.drop(columns=['year_y'])
if 'year_x' in merged_data.columns:
    merged_data = merged_data.rename(columns={'year_x': 'year'})

# Final dataset inspection
print("Final merged dataset shape:", merged_data.shape)
print("Final merged dataset preview:")
print(merged_data.head())

# Save the final merged dataset as a CSV file
merged_data.to_csv('final_merged_dataset.csv', index=False)

Final merged dataset shape: (1539, 20)
Final merged dataset preview:
   year    state state_po county_name     candidate       party  \
0  2012  Arizona       AZ      Apache  BARACK OBAMA    DEMOCRAT   
1  2012  Arizona       AZ     Cochise   MITT ROMNEY  REPUBLICAN   
2  2012  Arizona       AZ    Coconino  BARACK OBAMA    DEMOCRAT   
3  2012  Arizona       AZ        Gila   MITT ROMNEY  REPUBLICAN   
4  2012  Arizona       AZ      Graham   MITT ROMNEY  REPUBLICAN   

   candidatevotes  totalvotes      County Name State Name  Females 18-24 %  \
0           17147       25848    Apache County    Arizona         0.100623   
1           29497       49003   Cochise County    Arizona         0.081688   
2           29257       51731  Coconino County    Arizona         0.188882   
3           13455       21528      Gila County    Arizona         0.059434   
4            8076       11855    Graham County    Arizona         0.119623   

   Males 65+ %  White alone %  Black or African American al

In [152]:
from sqlalchemy import create_engine, text

# Database connection details
db_username = 'root'
db_password = 'election-project-data'
public_ip = '35.199.15.59'
db_name = 'outcomes-demographics'

# Create a database engine
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{public_ip}/{db_name}')

# Define a table creation query
create_table_query = """
CREATE TABLE outcomes_demographics (
    id INT AUTO_INCREMENT PRIMARY KEY,
    year INT,
    state VARCHAR(255),
    state_po VARCHAR(255),
    county_name VARCHAR(255),
    candidate VARCHAR(255),
    party VARCHAR(255),
    candidatevotes INT,
    totalvotes INT,
    state_name VARCHAR(255),
    males_18_24 FLOAT,
    females_65_plus FLOAT,
    white_alone FLOAT,
    black_or_african_american_alone FLOAT,
    less_than_39999 FLOAT,
    more_than_200000 FLOAT,
    regular_high_school_diploma FLOAT,
    masters_degree FLOAT,
    in_labor_force FLOAT,
    not_in_labor_force FLOAT
);
"""

# Execute the query to create the table
try:
    with engine.connect() as conn:
        conn.execute(text(create_table_query))
        print("Table 'outcomes_demographics' created or already exists.")
except Exception as e:
    print(f"Error creating table: {e}")

# Load the CSV data
merged_data = pd.read_csv('final_merged_dataset.csv')

Table 'outcomes_demographics' created or already exists.


In [ ]:
# Upload the data to the MySQL table
try:
    merged_data.to_sql('outcomes_demographics', con=engine, if_exists='replace', index=False)
    print("Data uploaded successfully to 'outcomes_demographics'.")
except Exception as e:
    print(f"Error uploading data: {e}")

# Connecting to MySQL database to validate outcomes_demographics table upload
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM outcomes_demographics;"))
        row_count = result.fetchone()[0]
        print(f"Number of rows in 'outcomes_demographics': {row_count}")
except Exception as e:
    print(f"Error verifying data: {e}")

Data uploaded successfully to 'outcomes_demographics'.
Number of rows in 'outcomes_demographics': 1539


In [153]:
# Validating Google Cloud bucket and connection
import os
from google.cloud import storage
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/ds-final-project-443519-b7e05f36c0a4.json'

# Initialize the client
client = storage.Client()

# List buckets
buckets = list(client.list_buckets())
print("Buckets in the project:")
for bucket in buckets:
    print(bucket.name)

Buckets in the project:
ds-final-project


In [156]:
def list_bucket_contents(bucket_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blobs = bucket.list_blobs()

    print(f"Files in bucket {bucket_name}:")
    for blob in blobs:
        print(blob.name)

# Call the function
list_bucket_contents('ds-final-project')

Files in bucket ds-final-project:
gs://ds-final-project/


In [157]:
# uploading merged_data.csv to google cloud bucket
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

upload_to_gcs('ds-final-project', 'final_merged_dataset.csv', 'final_merged_dataset.csv')

File final_merged_dataset.csv uploaded to final_merged_dataset.csv.


In [158]:
list_bucket_contents('ds-final-project')

Files in bucket ds-final-project:
final_merged_dataset.csv
gs://ds-final-project/
